In [14]:
import os
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Đường dẫn thư mục chứa dữ liệu
folder_path = "../Data_FTSE_model"

# Lấy danh sách tất cả file CSV trong thư mục
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Kiểm tra danh sách file
print("📂 Danh sách file CSV:", csv_files)


📂 Danh sách file CSV: ['AHT.csv', 'BARC.csv', 'BATS.csv', 'BP.csv', 'HLMA.csv', 'JMAT.csv', 'LGEN.csv', 'PSON.csv', 'REL.csv', 'RKT.csv', 'SDR.csv', 'SGE.csv', 'SHEL.csv', 'SSE.csv', 'STAN.csv', 'STJ.csv', 'TSCO.csv', 'VOD.csv']


In [ ]:
# Tham số cửa sổ trượt
train_size = 750
test_size = 250
step_size = 250  # Mỗi lần dịch cửa sổ

# Duyệt từng file để huấn luyện ARIMA và dự báo
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date")

    # Tổng số điểm dữ liệu
    total_points = len(df)

    # Lưu kết quả dự báo
    all_forecasts = []

    # Cửa sổ trượt
    for start in range(0, total_points - train_size - test_size + 1, step_size):
        train_end = start + train_size
        test_end = train_end + test_size

        # Tạo tập train và test
        train_data = df.iloc[start:train_end]["Close"]
        test_dates = df.index[train_end:test_end]

        try:
            # Huấn luyện mô hình ARIMA(p=1, d=1, q=1)
            model = ARIMA(train_data, order=(1,1,1))
            model_fit = model.fit()

            # Dự báo 250 ngày tiếp theo
            forecast = model_fit.forecast(steps=test_size)
            forecast.index = test_dates

            # Lưu kết quả vào danh sách
            forecast_df = pd.DataFrame({"Date": test_dates, "Predicted_Close": forecast.values})
            all_forecasts.append(forecast_df)

            print(f"✅ Hoàn thành cửa sổ {start} → {test_end} ({file})")

        except Exception as e:
            print(f"⚠️ Lỗi với {file} (từ {start}): {e}")

    # Gộp tất cả dự báo lại và lưu file
    if all_forecasts:
        final_forecast_df = pd.concat(all_forecasts)
        output_file = '../output/arima/ARIMA_' + file
        final_forecast_df.to_csv(output_file, index=False)
        print(f"📁 Dự báo hoàn thành: {output_file}")